<a href="https://colab.research.google.com/github/ParkJeongUng/Ung.github.io/blob/master/_notebooks/%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D_cross_validation_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98_GridSearchCV_%EB%9E%9C%EB%8D%A4%EC%84%9C%EC%B9%98_RandomizedSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증 cross validation //  그리드 서치 GridSearchCV // 랜덤서치 RandomizedSearchCV

## 검증 세트
* 훈련 세트를 한번 더 나눠서 테스트 세트 이외의 새로운 데이터 세트를 만든다.
* 만들어진 검증세트는 모델의 가장 좋은 매개변수를 찾는데 사용된다.

In [ ]:
# 데이터 준비|
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
wine_input = wine[['alcohol', 'sugar', 'pH']].to_numpy()
wine_target = wine['class'].to_numpy()

In [ ]:
# 훈련세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(wine_input, wine_target, test_size = 0.2, random_state = 42)

In [ ]:
# 나누어진 훈련세트를 이용하여 새로운 훈련세트와 검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [ ]:
# 나누어진 데이터 크기 확인
print(train_input.shape)
print(sub_input.shape, val_input.shape)

(5197, 3)
(4157, 3) (1040, 3)


In [ ]:
# 만들어진 새로운 훈련 세트로 훈련하고 검증세트로 평가하기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 훈련 세트에 과대적합된 모델 --> 좋은 하이퍼 파라미터를 찾아야함

0.9971133028626413
0.864423076923077


## 교차 검증 // cross validation --> 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다.
* 검증 세트를 만들었기에 훈련 세트의 데이터 수가 줄어듦
* 교차검증을 이용하면 안정적인 검증점수와 더 많은 데이터를 훈련에 사용할 수 있다.


In [ ]:
# cross_validate
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores) 

# fit_time = 모델 훈련하는데 걸린 시간
# score_time = 모델 검증하는데 걸린 시간
# test_score = 검증 점수

{'fit_time': array([0.00829268, 0.00734234, 0.00762033, 0.00749898, 0.00734591]), 'score_time': array([0.00101829, 0.00070906, 0.00075388, 0.00072241, 0.00079203]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
# 검증 점수의 평균
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# 회귀 모델일 경우 KFold()
# 분류 모델일 경우 StratifiedKFold()

# 분류 모델인 DecisionTreeClassifier 는 StratifiedKFold() 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(scores) 

{'fit_time': array([0.01132774, 0.00710869, 0.00750995, 0.00733685, 0.00716186]), 'score_time': array([0.00084209, 0.00075936, 0.00071144, 0.00073504, 0.00069094]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
# cross_validate() 는 훈련세트를 섞지 않는다 --> 우리는 train_test_split으로 이미 데이터를 섞어서 나눴기 때문에 상관없음 --> shuffle = True
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42) # n_splits --> 몇폴드 교차검증을 할지 결정 --> 기본은 5폴드
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score'])) 

0.8574181117533719


## 연습

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

wine_input = wine[['alcohol', 'sugar', 'pH']].to_numpy()
wine_target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(wine_input, wine_target, random_state = 42, test_size = 0.2)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state = 42, test_size = 0.2)

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

from sklearn.model_selection import cross_validate
score = cross_validate(dt, train_input, train_target)
print(score)

import numpy as np
print(np.mean(score['test_score']))

from sklearn.model_selection import StratifiedKFold
score = cross_validate(dt, train_input, train_target , cv = StratifiedKFold())

splitter = StratifiedKFold(n_splits=10, shuffle = True, random_state = 42)
score = cross_validate(dt, train_input, train_target, cv = splitter)
print(score)
print(np.mean(score['test_score']))

0.9971133028626413
0.864423076923077
{'fit_time': array([0.00733209, 0.00694227, 0.00748754, 0.00725961, 0.00699782]), 'score_time': array([0.0006938 , 0.00061035, 0.00068307, 0.00061774, 0.00060177]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
{'fit_time': array([0.00808716, 0.00804591, 0.00808191, 0.00787067, 0.00802708,
       0.00803208, 0.00794387, 0.00791359, 0.00801635, 0.00786281]), 'score_time': array([0.00054526, 0.00052047, 0.00048876, 0.00051832, 0.00057459,
       0.00050998, 0.00052834, 0.00052619, 0.00053453, 0.00055051]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
0.8574181117533719


## 그리드 서치
* 파라미터를 바꿔가면서 교차 검증을 수행한다
* 최적의 하이퍼 파라미터를 찾는다

In [ ]:
# 그리드 서치 
# 0.0001 ~ 0.0005 까지 0.0001 씩 증가하는 5개의 값을 시도
from sklearn.model_selection import GridSearchCV

# 파라미터를 딕셔너리 형태로 저장
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
# 객체 만들기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # n_jobs 는 cpu 코어 사용갯수 지정

# 최적의 하이퍼파라미터를 찾았으면 전체 훈련세트로 모델을 다시 만든다
gs.fit(train_input, train_target)

In [ ]:
# 그리드 서치는 모델 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 다시 모델을 훈련함
# 가장 높은 점수의 모델은 best_estimator_ 에 저장됨
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.9615162593804117
0.8653846153846154


In [ ]:
# 그리드 서치로 찾은 최적의 파라미터는 best_params_ 에 저장됨
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# 그리드 서치를 수행하면서 사용한 교차 검증의 점수는 cv_results_ 의 'mean_test_score' 에 저장됨
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
# 여러가지의 파라미터를 그리드 서치로 한 번에 찾기
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
          }

In [ ]:
# 그리드 서치 수행
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

In [ ]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
# 최상의 교차 검증 점수 확인
print(gs.cv_results_['mean_test_score'])
print(np.max(gs.cv_results_['mean_test_score']))

[0.85780355 0.85799604 0.85799604 ... 0.86126601 0.86165063 0.86357629]
0.8683865773302731


## 랜덤 서치
* 그리드 서치로 매개변수를 일일이 바꿔가며 모델을 훈련시켜 최적의 파라미터를 찾아내지 않을 수 있게 됨
* 그리드 서치는 매개변수의 범위를 내가 지정하는 거기 때문에 범위에 대한 근거가 부족함
* 랜덤 서치로 그 문제를 해결

In [ ]:
# 랜덤 서치는 매개변수 값의 목록을 전달하는게 아닌 매개변수를 샘플링 하는 확률 분포 객체를 전달함
from scipy.stats import uniform, randint # uniform 은 실수를 randint 는 정수를 뽑음

# 0 ~ 9 까지의 정수 중 10개를 랜덤 추출
rgen = randint(0, 10)
rgen.rvs(10)

array([3, 4, 8, 5, 2, 7, 3, 8, 6, 4])

In [ ]:
# np.unique 로 클래스를 확인하고 return_counts = True 로  클래스의 각 추출 된 횟수를 확인
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103, 101,  99,  99, 106, 108,  97,  94, 103,  90]))

In [ ]:
# 0 ~ 1 사이의 실수 중 10개를 랜덤 추출
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.50854909, 0.14208736, 0.22524214, 0.2296259 , 0.54808845,
       0.49409384, 0.7933543 , 0.07221253, 0.50430007, 0.18970072])

In [ ]:
# 파라미터에 확률 분포 전달하기
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25)}

In [ ]:
# 랜덤 서치 --> RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42) # n_iter --> 정의된 매개변수 범위에서 몇 번 샘플링 할 것인지 결정
rs.fit(train_input, train_target)

In [ ]:
# 최적의 파라미터 출력
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
# 최고의 교차검증 점수 출력
print(rs.cv_results_['mean_test_score'])
print(rs.cv_results_['mean_test_score'].shape)
print(np.max(rs.cv_results_['mean_test_score']))

[0.86511513 0.86261235 0.86838528 0.86588547 0.86376731 0.86434497
 0.86280503 0.86280484 0.86357592 0.86357555 0.86280503 0.8626142
 0.86472977 0.86954283 0.86203543 0.86761827 0.86222884 0.86473033
 0.86877082 0.86184423 0.86126657 0.86511494 0.8626142  0.86203543
 0.86511476 0.86607722 0.86222773 0.86684682 0.86261309 0.86338436
 0.8629977  0.86242171 0.86184478 0.86165211 0.86049808 0.86530706
 0.86280521 0.86684775 0.86203524 0.86318983 0.86780947 0.86761624
 0.86126694 0.86934867 0.86857889 0.86530743 0.86434497 0.86415303
 0.86838602 0.86530688 0.86145813 0.86684626 0.8618446  0.86145961
 0.86338454 0.86569131 0.86242152 0.86376805 0.86203543 0.86376916
 0.86511457 0.86184275 0.86338454 0.86242004 0.86107481 0.86203654
 0.86184478 0.86434552 0.86184478 0.86338473 0.86299993 0.8641534
 0.86338269 0.85972662 0.86415303 0.86665433 0.86261253 0.86222884
 0.86858111 0.86472903 0.86242097 0.86261457 0.86742448 0.86434497
 0.86684682 0.86184423 0.86107481 0.86877193 0.86338362 0.862421

In [ ]:
# 최적의 모델로 훈련하고 테스트 세트로 성능 확인
dt = rs.best_estimator_
dt.fit(train_input, train_target)
print(dt.score(test_input, test_target))

0.86
